# Chapter 3 The forecaster’s toolbox

## 3.1 Some simple forecasting methods

* Average 
* Naive : 마지막 관측값
  * works remarkably well for many economic and financial time series.
* Seasonal Naive : 마지막 시즌의 관측값
  * ex) 작년 2월의 값
![](./images/ch03/02.png)

* Drift
  * the average change seen in the historical data
  * drawing a line between the first and last observations, and extrapolating it into the future
![](./images/ch03/01.png)
![](./images/ch03/03.png)
  

## 3.2 Transformations and adjustments

* 4개의 방법으로 조정 
  *  simplify the patterns in the historical data 
    * by removing known sources of variation
    * by making the pattern more consistent across the whole data set

### Calendar adjustments
  * monthly 데이터를 다루는데 28일, 30일, 31일 달별로 다르다. 이를 미세 조정 
  * 더 simple pattern이 되며, 이는 forcast에 도움
  * 다른 예시로는 sales 데이터에서는, 월별로 다른 근무일을 고려

![](./images/ch03/04.png)


### Population adjustments
  * 전체 양보다는 N명당 
  * ex) 도시 내 병상 침대수, 전체 침대수량만 보면 늘었어도, 1000명당 침대수를 따지면 줄었다면, 이는 인구효과로 인한 착시므로 조정
  
### Inflation adjustments
  * 돈의 가치가 시간에 따라 변동하므로 이를 반영
  * ex) 10년 전의 1억이 지금의 1억과는 다르다. 
  * Price Index를 이용해서 조정 ex) x_t = y_t / (z_2000/z_t)

### Mathematical transformations 
  * data transformations : 
   * ex) square root, cubit root
   * Power transformation, ex) Box-Cox transformations
    * to stabilize variance, 
    * to make the data more normal distribution-like, 
    
![](./images/ch03/05.png)   

  * -5부터 5까지 lambda를 조정, 의뢰해서 optimal lambda를 찾아도 된다. 
![](./images/ch03/08.png)   
![](./images/ch03/06.png) 
![](./images/ch03/07.png) 

  * Back-transform : 원래의 scale로 돌리는 것 
  
![](./images/ch03/09.png)  


### Bias adjustments

* transformed된 데이터에서 일부 statistics를 구하면 오류(?)가 생긴다.
* ex) transformed된 데이터에서 mean을 구했다고 이것이 진정한 mean이 아니다. (실은 median에 가깝다)
  * transformed unit에서는 symmetric하게 되었겠지만, 원래 데이터는 skew된 assymmetric이기에 

![](./images/ch03/10.png) 
* https://www.rdocumentation.org/packages/forecast/versions/8.5/topics/rwf

## 3.3 Residual diagnostics

### Residuals

* 예측치와 실제의 차이, 즉 예측 오차 
* 적어도 다음 2개의 속성을 만족해야 하며, 아니라면 대응이 필요하다. 
  * residual끼리 uncorrelated => 아니라면.. Ch 9. 
  * zero mean =>아니라면, 예측에 bias가 있는 것이다. 예측에서 이 bias를 빼주는 것으로 수정

* 위의 두 속성을 만족하지 않는다면 개선이 필요한 forcast 방법이다. 하지만 만족한다고 해도 개선을 할 수는 있다. 

* 다음 속성도 만족하면 좋다. (예를 들어 prediction interval 구할 때)
  * constant variance
  * normal distribution
  
* 예시 : 구글의 daily 주가 
  * naive forcast 방법 사용했고, 이 방법을 평가하기 위해 residual의 속성을 조사해 보자 
  * residual이 zero mean이고 normal 하게 보이고, uncorrelated된 듯 하니 naive forcast도 괜찮다고 본다. 

![](./images/ch03/11.png) 
![](./images/ch03/12.png) 
![](./images/ch03/13.png) 
![](./images/ch03/14.png) 

### Residual의 ACF에서 auto-correlation을 더 정확하게 판별하기 위한 방안

* we test whether the first h autocorrelations are significantly different from what would be expected from a white noise process

#### Box-Pierce test
![](./images/ch03/15.png) 
* residual이 uncorrelated 되었다면, Q값이 작으면 된다. 

#### Ljung-Box test
![](./images/ch03/16.png)


### 위 두 테스트를 구글 주가 문제에 다시 적용하면

![](./images/ch03/17.png)

## 3.4 Evaluating forecast accuracy


### Training and test sets
* 앞선 구글 주가 문제에서는 forcast의 정확도를 forcast의 fit에 사용한 데이터를 재사용했으므로 정확한 과정이 아니다. 
* 즉 학습셋과 테스트셋의 분리가 필요하다. 
  * 테스트셋을 hold-out set, out-of-sample set이라고도 부른다. 

![](./images/ch03/18.png)

* 학습셋에 잘 fit되었다고 forcast를 잘 하는 것은 아니다. (일반화의 문제)
* perfect fit은 모델의 파라미터가 충분할 때 된다. 
* overfit도 발생

### Forecast errors

* 테스트셋에서의 예측과 실제의 차이 
* multi-step forcast시의 에러

![](./images/ch03/19.png)

* residual과의 차이 
  * residual은 training-set에서 정의 
  * residual은 one-step forcast
  
### Scale-dependent errors

* 해당 데이터의 unit으로 에러가 표현되는 경우 

![](./images/ch03/20.png)

### Percent Error

* unit-free error
  * p_t = 100* (e_t/y_t)

![](./images/ch03/21.png)

* unit이 다른 두 개의 time series forcast에 좋다. (ex. 화씨 온도의 기온 예측 v.s 섭씨 온도의 기온 예측)
* 단점도 있다. 
  * y_t가 0이면 Undefined, y_t가 0에 가까워도 문제
  * positive error보다 negative error에 더 weight을 준다(?)
  
### Scaled errors

* unit이 다른 forcast들을 비교하는 또다른 대안
* average of training error로 test error를 나눠준다고 생각하면 된다. 
* 아래 식에서는 naive method로 one-step forcast를 했을 때의 training avg error 구한 후 이를 나눠줌 
![](./images/ch03/22.png)


### 예시

![](./images/ch03/23.png)
![](./images/ch03/24.png)


### Time series cross-validation

* 파란 점은 training data, 빨간 점은 test data
* 모든 fold에서의 accuracy를 구하고, 최종은 이를 평균한다. 

![](./images/ch03/25.png)

* 아래는 4-step ahead forcast에 대한 CV

![](./images/ch03/26.png)

## 3.5 Prediction intervals

* forcast error가 normal하고, uncorrelated되어 있다고 가정한다면, 다음처럼 prediction interval을 표현 
  * 예를 들어 95% 신뢰구간이라면, c=1.96
![](./images/ch03/27.png)

### One-step prediction intervals

* std of forcast dist를 어떻게 구할 것인가?
* one-step forcast라고 한다면, 근사적으로 std of redisual dist과 같다고 본다. 
* 예를 들어, naive method를 쓸 때, 마지막 관측값이 531.48이었고, std of residual dist = 6.21 이라면

![](./images/ch03/28.png)

### Multi-step prediction intervals

* h가 커지면, 일반적으로 불확실해진다. 
* 역시 std of redisual dist를 사용해서 근사적으로 추정한다. 
![](./images/ch03/29.png)

![](./images/ch03/30.png)

### Prediction intervals from bootstrapped residuals

* 언제 사용?
  * 위의 방법들은 모두, forcast error dist이 redisual dist와 비슷하고, normal 분포일 것을 가정
  * forcast error dist가 실제로 normal이 아니면 어떻게 할 것인가?
* 어떻게?
  * 미래 관측열(forcast horizon)을 가상으로 수없이 생성해서 여기서 multi-step forcast error를 재고, 여기서 sigma 도출
  * 미래 관측열은 알려진 error (ex. one-step residual in training data)를 샘플링해서 가상으로 생성

![](./images/ch03/31.png) 
